In [16]:
import pandas as pd
import re
import string

from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from nltk import download
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
new_QA = pd.read_csv('QA_modified.csv', index_col=0)
new_QA

,question,content,category
id,,,
0,"Я сменил автомобить, на учет еще не поставил, ...",Для внесения данных по личному автомобилю обра...,автомобиль
1,Не отображается автомобиль в личном кабинете.,Для внесения данных по личному автомобилю обра...,автомобиль
2,добавить автомобиль,Для внесения данных по личному автомобилю обра...,автомобиль
3,хочу внести данные об автомобиле,Для внесения данных по личному автомобилю обра...,автомобиль
4,Как внести данные об автомобиле?,Для внесения данных по личному автомобилю обра...,автомобиль
...,...,...,...
1683,как убрать бота,Чат-бот находится в стадии пилотирования и обу...,поддержка
1684,мне сказали создать заявку в поддержке,Для оформления обращения в техническую поддерж...,оператор
1685,выход,Для оформления обращения в техническую поддерж...,оператор


In [3]:
download('stopwords')

morph = MorphAnalyzer()

stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(f"[{string.punctuation}]", "", text)  # Удаление пунктуации
    text = " ".join(morph.parse(word)[0].normal_form for word in text.split() if word not in stop_words)  # Лемматизация и удаление стоп-слов
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
new_QA['question_processed'] = new_QA['question'].apply(preprocess_text)
new_QA

,question,content,category,question_processed
id,,,,
0,"Я сменил автомобить, на учет еще не поставил, ...",Для внесения данных по личному автомобилю обра...,автомобиль,сменить автомобить учёт поставить мочь заправл...
1,Не отображается автомобиль в личном кабинете.,Для внесения данных по личному автомобилю обра...,автомобиль,отображаться автомобиль личный кабинет
2,добавить автомобиль,Для внесения данных по личному автомобилю обра...,автомобиль,добавить автомобиль
3,хочу внести данные об автомобиле,Для внесения данных по личному автомобилю обра...,автомобиль,хотеть внести дать автомобиль
4,Как внести данные об автомобиле?,Для внесения данных по личному автомобилю обра...,автомобиль,внести дать автомобиль
...,...,...,...,...
1683,как убрать бота,Чат-бот находится в стадии пилотирования и обу...,поддержка,убрать бот
1684,мне сказали создать заявку в поддержке,Для оформления обращения в техническую поддерж...,оператор,сказать создать заявка поддержка
1685,выход,Для оформления обращения в техническую поддерж...,оператор,выход


In [5]:
# Подсчет количества вхождений для каждого значения в колонке 'content'
content_counts = new_QA['content'].value_counts()

# Фильтрация строк, где значение 'content' встречается как минимум 3 раза
new_QA_filtered = new_QA[new_QA['content'].isin(content_counts[content_counts >= 3].index)]
new_QA_filtered.loc[1676, 'category'] = 'поддержка'


C:\Users\nikit\AppData\Local\Temp\ipykernel_1896\2523162306.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_QA_filtered.loc[1676, 'category'] = 'поддержка'


In [6]:
min_count = 2
category_counts = new_QA_filtered['category'].value_counts()
categories_with_one_record = category_counts[category_counts == 1].index
valid_categories = category_counts[category_counts >= min_count].index

single_record_df = new_QA_filtered[new_QA_filtered['category'].isin(categories_with_one_record)]
filtered_df = new_QA_filtered[new_QA_filtered['category'].isin(valid_categories)]

# Проверка категорий после фильтрации
print(f'Категории после фильтрации: {filtered_df["category"].value_counts()}')

# Разделение данных на 80% обучающих и 20% тестовых
train_df, test_df = train_test_split(filtered_df, test_size=0.2, stratify=filtered_df['category'], random_state=42)
train_df = pd.concat([train_df, single_record_df], axis=0)

print(f'Размер обучающей выборки: {len(train_df)}')
print(f'Размер тестовой выборки: {len(test_df)}')

# Проверка распределения категорий в обучающей и тестовой выборках
print(f'Категории в обучающей выборке: {train_df["category"].value_counts()}')
print(f'Категории в тестовой выборке: {test_df["category"].value_counts()}')

Категории после фильтрации: ЛК                     473
поддержка              283
табель                 168
удаленная работа        98
отпуск                  97
моя карьера             56
увольнение              49
отгул                   44
заявки                  40
зарплата                38
прием на работу         33
БиР                     31
график работы           28
ЭЦП                     22
документооборот         22
налоговый вычет         12
уход за больным          9
оператор                 8
автомобиль               8
справка                  8
обучение                 7
больничный               6
командировка             5
материальная помощь      3
доверенность             3
Name: category, dtype: int64
Размер обучающей выборки: 1241
Размер тестовой выборки: 311
Категории в обучающей выборке: ЛК                     378
поддержка              226
табель                 134
удаленная работа        78
отпуск                  78
моя карьера             45
увольнение     

In [7]:
train_df

,question,content,category,question_processed
id,,,,
1316,"У меня убрали в моих сервисах, роль _ Команда ...","Если в ""команде"" нет подчиненных сотрудников п...",табель,убрать мой сервис роль команда скидывать парол...
709,Как создать заявку на налоговый вычет?,"Для получения вычета создайте, пожалуйста, зая...",налоговый вычет,создать заявка налоговый вычет
512,Нет возможности зайти в ЛК- не верный логин ил...,"При проблемах со входом в личный кабинет, преж...",ЛК,возможность зайти лк верный логин пароль
1044,АДМ не может участвовать в программе моя карье...,"Создайте, пожалуйста, обращение в ИТ поддержку...",поддержка,адм участвовать программа карьера дмссылка отс...
221,Не могу закрыть заявку,"Первым делом, просьба очистить кэш/куки браузе...",ЛК,мочь закрыть заявка
...,...,...,...,...
47,хочу поменять магазин во время отпуска по бере...,Перевод возможен после окончания отпуска по бе...,БиР,хотеть поменять магазин время отпуск беременность
672,"Обучение по программе ""Моя карьера""","Доступ к программе ""карьера"" появляется спустя...",моя карьера,обучение программа карьера
300,неверно указан номер при приеме,"Кнопка ""изменить номер"" телефона находится в л...",ЛК,неверно указать номер приём


In [8]:
import random


def add_or_remove_punctuation(text):
    """Добавление или удаление знаков препинания."""
    # Возможные варианты добавления знаков препинания
    punctuations = [',', '.', '!', '?']
    words = text.split()

    # Добавляем или удаляем знаки препинания
    if random.random() < 0.5:
        # Добавить знак препинания
        position = random.randint(0, len(words) - 1)
        punct = random.choice(punctuations)
        words[position] = words[position] + punct
    else:
        # Удалить знак препинания, если он есть
        text = text.translate(str.maketrans('', '', string.punctuation))
        words = text.split()

    return ' '.join(words)


def introduce_typo(text):
    """Создание опечаток в тексте."""
    if not text:
        return text

    words = text.split()
    index = random.randint(0, len(words) - 1)
    word = words[index]

    # Опечатки: замена, пропуск или дублирование символов
    typo_type = random.choice(['swap', 'remove', 'duplicate'])

    if typo_type == 'swap' and len(word) > 1:
        # Меняем местами соседние буквы
        pos = random.randint(0, len(word) - 2)
        word = list(word)
        word[pos], word[pos + 1] = word[pos + 1], word[pos]
        words[index] = ''.join(word)

    elif typo_type == 'remove' and len(word) > 1:
        # Удаляем случайную букву
        pos = random.randint(0, len(word) - 1)
        words[index] = word[:pos] + word[pos + 1:]

    elif typo_type == 'duplicate':
        # Дублируем случайную букву
        pos = random.randint(0, len(word) - 1)
        words[index] = word[:pos] + word[pos] + word[pos:]

    return ' '.join(words)


def shuffle_words(text):
    """Перестановка порядка слов."""
    words = text.split()
    if len(words) > 1:
        random.shuffle(words)
    return ' '.join(words)


In [9]:
id_map_content = {}
for x in train_df['content']:
  id_map_content[x] = id_map_content.get(x, len(id_map_content))
train_df['content_id'] = train_df['content'].map(id_map_content)

In [10]:
id_map_categories = {}
for x in train_df['category']:
  id_map_categories[x] = id_map_categories.get(x, len(id_map_categories))
train_df['category_id'] = train_df['category'].map(id_map_categories)

In [12]:
from tqdm.notebook import tqdm
tqdm.pandas()

AUG_NUM = 30

def balance_dataset(qa_df):
    # Шаг 1: Найти самый частовстречаемый ответ
    max_count = qa_df['content_id'].value_counts().max()

    # Шаг 2: Сбалансировать выборку ответов
    augmented_data = []

    for content_id, group in tqdm(qa_df.groupby('content_id')):
        count = len(group)
        augmented_data.extend(group.to_dict('records'))  # Добавляем все исходные строки

        # Если ответ встречается реже, чем самый частовстречаемый, создаем аугментированные копии вопросов
        for _ in range(min(AUG_NUM, max_count - count)):
            row = group.sample(1).iloc[0].to_dict()  # Случайный вопрос из группы
            question = row['question']

            # Применяем несколько аугментаций последовательно
            augmented_question = add_or_remove_punctuation(question)
            augmented_question = introduce_typo(augmented_question)
            augmented_question = shuffle_words(augmented_question)

            # Сохраняем аугментированный вопрос с исходным ответом
            new_row = row.copy()
            new_row['question'] = augmented_question
            augmented_data.append(new_row)

    # Шаг 3: Создать новый сбалансированный датафрейм
    balanced_qa_df = pd.DataFrame(augmented_data)
    return balanced_qa_df

balanced_train_QA = balance_dataset(train_df)
balanced_train_QA

  0%|          | 0/100 [00:00<?, ?it/s]

,question,content,category,question_processed,content_id,category_id
0,"У меня убрали в моих сервисах, роль _ Команда ...","Если в ""команде"" нет подчиненных сотрудников п...",табель,убрать мой сервис роль команда скидывать парол...,0,0
1,с 1 августа был перевод на должность директора...,"Если в ""команде"" нет подчиненных сотрудников п...",табель,1 август перевод должность директор магазин ли...,0,0
2,В личном кабинете отсутвует список подчиненных,"Если в ""команде"" нет подчиненных сотрудников п...",табель,личный кабинет отсутвовать список подчинённый,0,0
3,не могу создать заявку на сотрудника в личном ...,"Если в ""команде"" нет подчиненных сотрудников п...",табель,мочь создать заявка сотрудник личный кабинет,0,0
4,На портале пропали все разделы создания заявок...,"Если в ""команде"" нет подчиненных сотрудников п...",табель,портал пропасть раздел создание заявка сотрудник,0,0
...,...,...,...,...,...,...
4183,моя направляется дату зарплата В присттавам какую,"Обращаем внимание, что работодатель производит...",зарплата,какой дата зарплата направляться пристав,99,10
4184,судебным удержанные Когда дененжые направляютс...,"Обращаем внимание, что работодатель производит...",зарплата,удержать денежный средство направляться судебн...,99,10
4185,ВВ зарплата моя направляется какую дату приставам,"Обращаем внимание, что работодатель производит...",зарплата,какой дата зарплата направляться пристав,99,10
4186,Когда денежные удержанные судебным средства пп...,"Обращаем внимание, что работодатель производит...",зарплата,удержать денежный средство направляться судебн...,99,10


In [14]:
# Преобразование текста в Bag of Words
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(balanced_train_QA['question_processed'])
X_test = vectorizer.transform(test_df['question_processed'])

y_train = balanced_train_QA['content']
y_test = test_df['content']

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [17]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.6945337620578779
Precision: 0.7301067635472781
Recall: 0.6945337620578779
F1 Score: 0.6986688121790211


C:\Users\nikit\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nikit\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# Преобразование текста в TF-IDF
vectorizer_tf = TfidfVectorizer()
X_train_tf = vectorizer_tf.fit_transform(balanced_train_QA['question_processed'])
X_test_tf = vectorizer_tf.transform(test_df['question_processed'])

model2 = LogisticRegression()
model2.fit(X_train_tf, y_train)

y_pred_2 = model2.predict(X_test_tf)

In [20]:
accuracy = accuracy_score(y_test, y_pred_2)
precision = precision_score(y_test, y_pred_2, average='weighted')
recall = recall_score(y_test, y_pred_2, average='weighted')
f1 = f1_score(y_test, y_pred_2, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.7138263665594855
Precision: 0.7301749899168145
Recall: 0.7138263665594855
F1 Score: 0.7038205570939289


C:\Users\nikit\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nikit\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
